In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import joblib

In [2]:

def getData_half(cfileName):
    """
    Reads only the first half of the complex samples from a .cfile.
    The file is assumed to contain interleaved float32 I and Q samples.
    """
    # Get file size in bytes
    file_size = os.path.getsize(cfileName)
    # Determine total number of float32 samples in the file
    num_floats = file_size // 4
    # Each complex sample requires 2 floats.
    # The full complex array would have num_floats//2 elements.
    # To get only half of that, we need to read:
    count = (num_floats // 4) * 2  # (num_floats//4 complex samples)
    
    # Read only 'count' float32 numbers from the file
    data = np.fromfile(cfileName, dtype="float32", count=count)
    
    # Form complex samples: every pair (I, Q)
    data_complex = data[0::2] + 1j * data[1::2]
    
    return data_complex



In [3]:
# number of samples per class
num_samp_per_class = 10000

# FFT size for the STFT operation (which is same as the feature vector size)
fft_size = feature_vector_size = 2048

# number of overlapping samples for the STFT operation
fft_overlap = 256

In [4]:
class_label = 0

In [5]:
data = getData_half("/media/rasi/Other/data/API-Security/hypothesis1/autheticated_firemware/original")
print(data.shape)

(212153772,)


In [6]:
f, t, Zxx = signal.stft(data, fs=20e6, nperseg=fft_size, noverlap=fft_overlap)
print(len(f))
print(len(t))
print(Zxx.shape)

/tmp/ipykernel_111398/3323255673.py:1: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = signal.stft(data, fs=20e6, nperseg=fft_size, noverlap=fft_overlap)


2048
118391
(2048, 118391)


In [7]:
del data

Zxx = Zxx.transpose()

X = Zxx[:num_samp_per_class]
y = np.full(num_samp_per_class, class_label)
del Zxx

print(X.shape)
print(y.shape)

(10000, 2048)
(10000,)


In [8]:
class_label = 1 #remove random API

In [9]:
data = getData_half("/media/rasi/Other/data/API-Security/hypothesis1/unauthenticated_firemware/update_weatherRequest_different_payload")
print(data.shape)

(212010598,)


In [10]:
f, t, Zxx = signal.stft(data, fs=20e6, nperseg=fft_size, noverlap=fft_overlap)
print(len(f))
print(len(t))
print(Zxx.shape)

/tmp/ipykernel_111398/3323255673.py:1: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = signal.stft(data, fs=20e6, nperseg=fft_size, noverlap=fft_overlap)


2048
118311
(2048, 118311)


In [11]:
del data

Zxx = Zxx.transpose()

X = np.concatenate((X, Zxx[:num_samp_per_class]), axis=0) 
y = np.concatenate((y, np.full(num_samp_per_class, class_label)), axis=0)
del Zxx

print(X.shape)
print(y.shape)

(20000, 2048)
(20000,)


In [12]:
X = np.abs(X)
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [13]:
# Split data to train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [14]:
clf = SVC()
clf.fit(X_train, y_train)
# Save the model
joblib.dump(clf, "/media/rasi/Other/data/model/svm_update.pkl")  # Saves as a .pkl file
print("Model saved successfully!")

Model saved successfully!


In [15]:

clf = joblib.load("/media/rasi/Other/data/model/svm_update.pkl")
y_pred = clf.predict (X_test)
# Evaluate accuracy
print("SVM Accuracy:", accuracy_score(y_test, y_pred))

SVM Accuracy: 0.9805


In [16]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[972  13]
 [ 26 989]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       985
           1       0.99      0.97      0.98      1015

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000

